In [1]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print("Import successful!!")

2025-06-04 10:50:45.908090: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 10:50:45.922118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749027045.936321  334615 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749027045.940869  334615 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 10:50:45.957253: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Import successful
Import successful!!


In [2]:
# Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [3]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = {"paragraphs":[]}
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            paragraphs_text["paragraphs"].append(p.text.strip()) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [18]:
test_speech = con.sql("select * from speech order by random() limit 1 ").fetchone()

In [19]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

121955
{'paragraphs': ["Herr Präsident! Meine Damen und Herren! Ich darf mich auf meinen schriftlich vorliegenden Bericht beziehen und dazu nur noch zwei ergänzende Bemerkungen machen. Zunächst möchte ich zu § 35 a folgendes hinzufügen. Dieser Paragraph bestimmt, daß Arzneimittel, die aus Stoffen unbekannter Wirksamkeit und aus Zusammensetzungen von Stoffen unbekannter Wirksamkeit bestehen, drei Jahre unter die Rezeptpflicht fallen. Hiervon sind nicht nur dann Ausnahmen möglich, wenn die Wirkung neuer Zusammensetzungen voraussehbar ist; vielmehr sind von vornherein von dieser automatischen Rezeptpflicht für drei Jahre auch Zusammensetzungen aus Stoffen bisher unbekannter Wirksamkeit ausgenommen, wenn sie bisher frei verkäuflich waren. Dann möchte ich noch § 53 Abs. 1 näher erläutern. Abs. 1 betrifft den Fall, daß, wenn die neuen Voraussetzungen für die Herstellung von Arzneimitteln für den Herstellungsleiter nicht vorliegen, der Betrefmende aber -bei Inkrafttreten des Gesetzes mindeste

In [20]:
predict_batch(paragraphs)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

[[('Herr', 'B-EPPOL'),
  ('Präsident', 'I-EPPOL'),
  ('!', 'O'),
  ('Meine', 'O'),
  ('Damen', 'O'),
  ('und', 'O'),
  ('Herren', 'O'),
  ('!', 'O'),
  ('Ich', 'O'),
  ('darf', 'O'),
  ('mich', 'O'),
  ('auf', 'O'),
  ('meinen', 'O'),
  ('schriftlich', 'O'),
  ('vorliegenden', 'O'),
  ('Bericht', 'O'),
  ('beziehen', 'O'),
  ('und', 'O'),
  ('dazu', 'O'),
  ('nur', 'O'),
  ('noch', 'O'),
  ('zwei', 'O'),
  ('ergänzende', 'O'),
  ('Bemerkungen', 'O'),
  ('machen', 'O'),
  ('.', 'O'),
  ('Zunächst', 'O'),
  ('möchte', 'O'),
  ('ich', 'O'),
  ('zu', 'O'),
  ('§', 'O'),
  ('35', 'O'),
  ('a', 'O'),
  ('folgendes', 'O'),
  ('hinzufügen', 'O'),
  ('.', 'O'),
  ('Dieser', 'O'),
  ('Paragraph', 'O'),
  ('bestimmt', 'O'),
  (',', 'O'),
  ('daß', 'O'),
  ('Arzneimittel', 'O'),
  (',', 'O'),
  ('die', 'O'),
  ('aus', 'O'),
  ('Stoffen', 'O'),
  ('unbekannter', 'O'),
  ('Wirksamkeit', 'O'),
  ('und', 'O'),
  ('aus', 'O'),
  ('Zusammensetzungen', 'O'),
  ('von', 'O'),
  ('Stoffen', 'O'),
  ('unbeka

In [ ]:
print(ET.tostring(root.find('name'), encoding='unicode'))